In [ ]:
import os
from os.path import join, dirname, abspath
import sys
CURRENT_DIR = os.getcwd()
sys.path.insert(0, join(CURRENT_DIR, '../..'))  # Import local models

from cliport.environments.environment import Environment
import torch
from agents.PickPlaceAgent import PickPlaceAgent
from cliport import tasks
from cliport.dataset import RavensDataset
import numpy as np
from cliport.utils import utils


In [3]:
record_cfg = {
  "save_video": False,
  "save_video_path": "/home/ubuntu/VLM/videos/",
  "add_text": True,
  "fps": 20,
  "video_height": 640,
  "video_width": 720,
}

assets_root = "/home/ubuntu/cliport/cliport/environments/assets/"

env = Environment(
    assets_root,
    disp=False,
    shared_memory=False,
    hz=480,
    record_cfg=record_cfg
)

text argument:/home/ubuntu/cliport/cliport/environments/assets/
int args: [

In [9]:
agent = PickPlaceAgent(num_rotations=12, lr=1e-4, device='cuda')
agent.pick_model.load_state_dict(torch.load("/home/ubuntu/VLM/checkpoints/checkpoint_PairPack_epoch30.pth")['pick_state_dict'])
agent.place_model.load_state_dict(torch.load("/home/ubuntu/VLM/checkpoints/checkpoint_PairPack_epoch30.pth")['place_state_dict'])

<All keys matched successfully>

In [7]:
train_dataset_cfg = {"dataset":{"type": "single",
                    "images": True,
                    "cache": False,
                    "augment":{"theta_sigma":60},
                    "cache_size": 350},
                    }

# load data
train_dataset = RavensDataset('/home/ubuntu/cliport/data/packing-boxes-pairs-full-val', train_dataset_cfg, n_demos=100, augment=False)

In [ ]:
total_reward = 0
num_trials = 100
for i in [1,3]:#range(num_trials):
    env.start_rec("pick-place-pairs" + str(i))

    episode, seed = train_dataset.load(i)
    task = tasks.names["packing-boxes-pairs-full"]()
    task.mode = "train"
    env.seed(seed)
    env.set_task(task)
    obs = env.reset()
    info = env.info
    episode_reward = 0

    lang_goal = info['lang_goal']
    print(f'Lang Goal: {lang_goal}')
    
    for _ in range(10):
        img = train_dataset.get_image(obs)
        lang_goal = info['lang_goal']
        act, affordances = agent.act(img, lang_goal)

        obs, reward, done, info = env.step(act)
        episode_reward += reward
        total_reward += reward
        print("Single timestep reward:", reward)
        if done:
            break
    print("Iteration:", i)
    print("Episode Reward:", episode_reward)
    print("Average Reward Across Episodes:", total_reward/(i+1))
    env.end_rec()


In [11]:
env.end_rec()